In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("cumulative.csv")
df = df.drop(columns=["rowid", "kepid", "kepoi_name", "kepler_name", "koi_pdisposition", "koi_score", "koi_tce_delivname"])
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,FALSE POSITIVE,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,CONFIRMED,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


# Create a Train Test Split

Use `koi_disposition` for the y values

In [3]:
from sklearn.model_selection import train_test_split
y = df["koi_disposition"]
X = df.drop(columns=["koi_disposition"])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [4]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6033,0,0,0,0,219.931324,0.006700,-0.006700,329.38360,0.02260,-0.02260,...,-83.0,4.500,0.038,-0.113,0.912,0.116,-0.053,295.18680,39.575691,14.254
9329,1,0,0,0,34.998908,0.000350,-0.000350,160.32308,0.00739,-0.00739,...,-134.0,4.653,0.025,-0.053,0.648,0.066,-0.044,282.90509,43.871769,15.326
1661,0,0,0,0,2.892217,0.000007,-0.000007,132.87726,0.00192,-0.00192,...,-84.0,4.655,0.025,-0.020,0.615,0.025,-0.031,294.70938,45.969501,15.068
6308,1,0,0,0,386.621890,0.023550,-0.023550,319.12900,0.02680,-0.02680,...,-176.0,4.412,0.073,-0.274,1.033,0.411,-0.115,283.89960,45.724232,13.262
3008,0,0,0,0,9.378917,0.000022,-0.000022,176.45197,0.00188,-0.00188,...,-170.0,4.493,0.112,-0.138,0.808,0.162,-0.108,286.96390,37.829708,14.720


# Pre-processing

Scale the data using the MinMaxScaler

In [5]:
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)

X_train_scaled = X_minmax.transform(X_train)
X_test_scaled = X_minmax.transform(X_test)

# Train the Support Vector Machine

In [8]:
import numpy as np
from sklearn.svm import SVC 
model2 = SVC(kernel='linear')
model2.fit(X_train_scaled,np.ravel(y_train))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [9]:
print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

Training Data Score: 0.8587984141506557
Testing Data Score: 0.8499542543458372


# Hyperparameter Tuning

Use `GridSearchCV` to tune the `C` and `gamma` parameters

In [10]:
# Create the GridSearchCV model
model2 = SVC(kernel='linear')
model2.fit(X_test_scaled,np.ravel(y_test))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [16]:
# Train the model with GridSearch
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [50,250, 500,1000,2000],
              'gamma': [50, 25, 10,5, 1, 0.1]}
grid = GridSearchCV(model, param_grid, verbose=2)
grid.fit(X_train_scaled,np.ravel(y_train))

D:\Programs\Anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] C=50, gamma=50 ..................................................
[CV] ................................... C=50, gamma=50, total=   0.6s
[CV] C=50, gamma=50 ..................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV] ................................... C=50, gamma=50, total=   0.6s
[CV] C=50, gamma=50 ..................................................
[CV] ................................... C=50, gamma=50, total=   0.5s
[CV] C=50, gamma=25 ..................................................
[CV] ................................... C=50, gamma=25, total=   0.6s
[CV] C=50, gamma=25 ..................................................
[CV] ................................... C=50, gamma=25, total=   0.6s
[CV] C=50, gamma=25 ..................................................
[CV] ................................... C=50, gamma=25, total=   0.5s
[CV] C=50, gamma=10 ..................................................
[CV] ................................... C=50, gamma=10, total=   0.6s
[CV] C=50, gamma=10 ..................................................
[CV] ................................... C=50, gamma=10, total=   0.5s
[CV] C=50, gamma=10 ..................................................
[CV] .

[CV] ................................. C=1000, gamma=25, total=   2.4s
[CV] C=1000, gamma=10 ................................................
[CV] ................................. C=1000, gamma=10, total=   2.5s
[CV] C=1000, gamma=10 ................................................
[CV] ................................. C=1000, gamma=10, total=   3.1s
[CV] C=1000, gamma=10 ................................................
[CV] ................................. C=1000, gamma=10, total=   2.4s
[CV] C=1000, gamma=5 .................................................
[CV] .................................. C=1000, gamma=5, total=   2.5s
[CV] C=1000, gamma=5 .................................................
[CV] .................................. C=1000, gamma=5, total=   3.2s
[CV] C=1000, gamma=5 .................................................
[CV] .................................. C=1000, gamma=5, total=   2.6s
[CV] C=1000, gamma=1 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:  3.0min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='linear',
                           max_iter=-1, probability=False, random_state=None,
                           shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [50, 250, 500, 1000, 2000],
                         'gamma': [50, 25, 10, 5, 1, 0.1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [17]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 2000, 'gamma': 50}
0.8888380603842635
